In [1]:
from argparse import ArgumentParser
import utils
import torch
from models.basic_model import CDEvaluator
import os
import numpy as np
import pandas as pd
from typing import TypedDict
import xarray as xr
import rioxarray as rxr
import rasterio as rio
from rasterio.enums import Resampling
import glob
import certifi
import pyproj
import pystac
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=4)
client = Client(cluster)
print(f"/proxy/{client.scheduler_info()['services']['dashboard']}/status")

/proxy/8787/status


2022-06-06 00:34:09,972 - distributed.diskutils - INFO - Found stale lock file and directory '/home/dylan/code/planet-regrid-poc/SiameseCD/dask-worker-space/worker-h5mnnbsg', purging
2022-06-06 00:34:09,973 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [2]:
def get_args():
    # ------------
    # args
    # ------------
    parser = ArgumentParser()
    parser.add_argument('--project_name', default='BIT_LEVIR', type=str)
    parser.add_argument('--gpu_ids', type=str, default='0', help='gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU')
    parser.add_argument('--checkpoint_root', default='checkpoints', type=str)
    parser.add_argument('--output_folder', default='samples/predict', type=str)

    # data
    parser.add_argument('--num_workers', default=0, type=int)
    parser.add_argument('--dataset', default='CDDataset', type=str)
    parser.add_argument('--data_name', default='quick_start', type=str)

    parser.add_argument('--batch_size', default=1, type=int)
    parser.add_argument('--split', default="demo", type=str)
    parser.add_argument('--img_size', default=256, type=int)

    # model
    parser.add_argument('--n_class', default=2, type=int)
    parser.add_argument('--net_G', default='base_transformer_pos_s4_dd8_dedim8', type=str,
                        help='base_resnet18 | base_transformer_pos_s4_dd8 | base_transformer_pos_s4_dd8_dedim8|')
    parser.add_argument('--checkpoint_name', default='best_ckpt.pt', type=str)

    args = parser.parse_args(args=[])
    return args

In [8]:
def load_model():
    args = get_args()
    # device = torch.device("cuda:%s" % args.gpu_ids[0]
                          # if torch.cuda.is_available() and len(args.gpu_ids)>0
                        # else "cpu")
    # args.checkpoint_dir = os.path.join(args.checkpoint_root, args.project_name)
    # os.makedirs(args.output_folder, exist_ok=True)

    # log_path = os.path.join(args.output_folder, 'log_vis.txt')

    # data_loader = utils.get_loader(args.data_name, img_size=args.img_size,
    #                                batch_size=args.batch_size,
    #                                split=args.split, is_train=False)

    model = CDEvaluator(args)
    model.load_checkpoint(args.checkpoint_name) 
    model.eval()
    return model

In [9]:
remote_model = client.submit(load_model)
print(remote_model)

<Future: pending, key: load_model-b9b60b69a4cfb6fd8005b0956b529b7f>


2022-06-06 00:35:47,326 - distributed.worker - WARNING - Compute Failed
Key:       load_model-b9b60b69a4cfb6fd8005b0956b529b7f
Function:  load_model
args:      ()
kwargs:    {}
Exception: 'AttributeError("\'Namespace\' object has no attribute \'checkpoint_dir\'")'



# Load Planet Data

In [ ]:
#os.environ['PROJ_LIB'] = '/opt/conda/bin/proj'

In [ ]:
a = rio.vrt.WarpedVRT(rio.open('./catalogs/21_05/mosaic.tif'))
b = rio.vrt.WarpedVRT(rio.open('./catalogs/22_05/mosaic/mosaic.tif'),transform=a.transform,height=a.height,width=a.width)

ds1 = rxr.open_rasterio(a,chunks=(4,8192,8192),lock=False)
ds2 = rxr.open_rasterio(b,chunks=(4,8192,8192),lock=False)#'./catalogs/22_05/mosaic/mosaic.tif'

# upscale_factor = 3
# w_ds1 = ds1.rio.width*upscale_factor
# h_ds1 = ds1.rio.height*upscale_factor
# w_ds2 = ds2.rio.width*upscale_factor
# h_ds2 = ds2.rio.height*upscale_factor

# ds1 = ds1.rio.reproject(ds1.rio.crs,
#                         shape=(h_ds1,w_ds1),
#                         resampling=Resampling.bilinear,
#                        )

# ds2 = ds2.rio.reproject(ds2.rio.crs,
#                         shape=(h_ds2,w_ds2),
#                         resampling=Resampling.bilinear,
#                        )

ds1 = ds1[:3]
ds2 = ds2[:3]

ds1 = ds1/255.0
ds2 = ds2/255.0

# m1 = ds1.mean(axis=[1,2])
# s1 = ds1.std(axis=[1,2])
# m2 = ds2.mean(axis=[1,2])
# s2 = ds2.std(axis=[1,2])

In [ ]:
ds = xr.combine_nested([ds1,ds2],concat_dim="time").chunk((2,3,256,256))

In [ ]:
bands = xr.DataArray([1,2,3],name="band",dims=["band"],coords={"band":[1,2,3]})
first_mu = xr.DataArray(np.array([0.5,0.5,0.5],dtype="float32"),name="mean",coords=[bands])
second_mu = xr.DataArray(np.array([0.5,0.5,0.5],dtype="float32"),name="mean",coords=[bands])
first_std = xr.DataArray(np.array([0.5,0.5,0.5],dtype="float32"),name="std",coords=[bands])
second_std = xr.DataArray(np.array([0.5,0.5,0.5],dtype="float32"),name="std",coords=[bands])
# first_mu = xr.DataArray(m1.data,name="mean",coords=[bands])
# first_std = xr.DataArray(s1.data,name="std",coords=[bands])
# second_mu = xr.DataArray(m2.data,name="mean",coords=[bands])
# second_std = xr.DataArray(s2.data,name="std",coords=[bands])

mean = xr.concat([first_mu,second_mu],dim="time")
std = xr.concat([first_std,second_std],dim="time")

# ds = ds/255.0
normalized = (ds-mean)/std

normalized = normalized.chunk((2,3,256,256))
normalized

In [ ]:
slices = {}
for coord in ["y","x"]:
    remainder = len(ds.coords[coord])%32
    slice_ = slice(-remainder) if remainder else slice(None)
    slices[coord] = slice_

ds_comb = normalized.isel(**slices)

ds_comb

In [ ]:
import dask.array
def predict_chips(data,model)->torch.Tensor:
    result = model._forward_pass(data).cpu().numpy()[0][0]
    return result

def copy_and_predict_chunked(tile,model,token=None):
    slices = dask.array.core.slices_from_chunks(dask.array.empty(tile.shape).chunks)
    out = np.empty(shape=tile.shape[2:], dtype="uint8")
    device = torch.device("cuda")
    A = torch.as_tensor(tile[0][np.newaxis, ...]) #gpu_chip = torch.as_tensor(tile[slice_][np.newaxis, ...]).to(device)
    B = torch.as_tensor(tile[1][np.newaxis, ...])
    gpu_chip = {'name':'test','A':A.float().to(device),'B':B.float().to(device),'L':torch.zeros(1,1,256,256).float().to(device)}
    out = predict_chips(gpu_chip, model)
    return out

In [ ]:
meta = np.array([[]], dtype="uint8")[:0]

predictions_array = ds_comb.data.map_blocks(
    copy_and_predict_chunked,
    meta=meta,
    drop_axis=[0,1],
    model=remote_model,
    name="predict",
)

predictions_array

predictions = xr.DataArray(
    predictions_array,
    coords=ds_comb.drop_vars("band").coords,
    dims=("y", "x"),
)
predictions

In [ ]:
test = predictions.compute()

In [ ]:
import matplotlib.colors
from bokeh.models.tools import BoxZoomTool
import panel
import hvplot.xarray

def logo(plot, element):
    plot.state.toolbar.logo = None


zoom = BoxZoomTool(match_aspect=True)
style_kwargs = dict(
    width=450,
    height=400,
    xaxis=False,
    yaxis=False,
)
kwargs = dict(
    x="x",
    y="y",
    rasterize=True,
    cmap='gray',
    aggregator="mode",
    colorbar=False,
    tools=["pan", zoom, "wheel_zoom", "reset"],
)

In [ ]:
middle = ds.shape[2] // 2, ds.shape[3] // 2
slice_y = slice(middle[0], middle[0] + 5_000)
slice_x = slice(middle[1], middle[1] + 5_000)

parts = [x.isel(y=slice_y, x=slice_x) for x in [ds, predictions, test]]

In [ ]:
ds_local, predictions_local, test_local = dask.compute(*parts)

In [ ]:
panel.Column(
    panel.Row(
        ds_local.sel(time=0)
        .hvplot.rgb(
            bands="band", rasterize=True, hover=False, title="Austin 05_21", tools=["pan", zoom, "wheel_zoom", "reset"], **style_kwargs
        )
        .opts(default_tools=[], hooks=[logo]),
        test_local
        .hvplot.image(title="Changes", **kwargs, **style_kwargs)
        .opts(default_tools=[]),
        ds_local.sel(time=1)
        .hvplot.rgb(
            bands="band",
            rasterize=True,
            hover=False,
            title="Austin 05_22",
            tools=["pan", zoom, "wheel_zoom", "reset"],
            **style_kwargs,
        )
        .opts(default_tools=[], hooks=[logo]),
    ),
)